In [1]:
import pandas as pd
import pickle

In [2]:
def load_data(movie_df_path, poster_df_path) -> pd.DataFrame():
    """
    Добавляет постеры к фильмам к датафрейму с фильмами
    """

    with (open(movie_df_path, "rb")) as f:
        movie_df = pickle.load(f)

    # Load poster dataframe from csv
    posters_df = pd.read_csv(poster_df_path, sep="\t")

    # Prepare poster dataframe
    # Fill NaN values with empty string
    posters_df.fillna("", inplace=True)
    # Remove year from the title (Fight Club (1999) -> Fight Club)
    posters_df["title"] = posters_df["title"].apply(lambda x: x[:-7])
    # Remove "The" from the end of the title (Shaw shank Redemption, The -> Shaw shank Redemption)
    posters_df["title"] = posters_df["title"].apply(lambda x: x[:-5] if x.endswith(", The") else x)

    # Merge two dataframes by title
    df = movie_df.merge(posters_df, on="title", how="left")

    # Remove rows with no poster (None values)
    df = df[df["poster_link"].notna()]
    # Drop year_y column
    df.drop(columns=["year_y"], inplace=True)
    # Rename year_x to year
    df.rename(columns={"year_x": "year"}, inplace=True)
    # Drop item column
    df.drop(columns=["item"], inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)

    return df

In [3]:
movies_df = load_data("../data/datasets/movies_df.pickle", "../data/datasets/poster.csv")

In [4]:
# Добавляем к фильмам оценки пользователей. Для этого загружаем датасеты с фильмами, рейтингами и ссылками между ними
def add_rating(movies_df: pd.DataFrame, ratings_path: str, links_path: str):
	"""
	Добавляет оценки пользователей к датафрейму с фильмами
	"""
	
	# Загружаем датасеты
	rating_df = pd.read_csv(ratings_path)
	links_df = pd.read_csv(links_path)

	# Переименовываем столбец с id фильма в обоих датасетах
	movies_df.rename(columns={"id": "tmdb_id"}, inplace=True)
	links_df.rename(columns={"tmdbId": "tmdb_id"}, inplace=True)

	# Объединяем датасеты по id фильма
	tmp_df = movies_df.merge(links_df, on='tmdb_id')

	# Удаляем дубликаты и пропущенные значения
	tmp_df = tmp_df.drop_duplicates(subset="tmdb_id")
	tmp_df = tmp_df.dropna(subset="movieId")

	# Объединяем датасеты по id фильма
	df = rating_df.merge(tmp_df, on='movieId')

	return df

In [5]:
df = add_rating(movies_df, "../data/datasets/ratings.csv", "../data/datasets/links.csv")

In [7]:
with open('../data/datasets/movies.pickle', 'wb') as f:
	pickle.dump(df, f)

In [8]:
# Удаляем более не нужные csv файлы
import os
#os.remove("../data/datasets/ratings.csv")
#os.remove("../data/datasets/links.csv")
#os.remove("../data/datasets/poster.csv")
#os.remove("../data/datasets/movies_df.pickle")